In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from transformers import BertTokenizer
import torch
from torch import nn
from transformers import BertModel
import numpy as np
from torch.optim import Adam
from tqdm import tqdm

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory


import os

df = pd.read_csv("/kaggle/input/feedback-prize-2021/train.csv")
df

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
labels = {'Lead':0,
          'Position':1,
          'Claim':2,
          'Concluding Statement':3,
          'Evidence':4,
          'Counterclaim':5,
          'Rebuttal':6
          }
prev_label = {'Lead':0,
          'Position':1,
          'Claim':2,
          'Concluding Statement':3,
          'Evidence':4,
          'Counterclaim':5,
          'Rebuttal':6,
          'Start':7
          }

In [3]:
sequence = df['predictionstring']
sequence = sequence.apply(lambda x: x.split())
sequence = sequence.apply(lambda x: list(map(int, x)))
sequence
df_new = df
start, end = [],[]
for i in range(len(sequence)):
    start.append(sequence[i][0])
    end.append(sequence[i][-1])
df_new['Start'] = start
df_new['End'] = end

prev = [7]

for i in range(1, len(sequence)):
    if df_new['End'][i-1] < df_new['Start'][i]:
        prev.append(prev_label[df['discourse_type'][i-1]])
    else:
        prev.append(7)

In [4]:
df_new['one_hot'] = prev
df_new

In [5]:
from gensim.utils import simple_preprocess
# Tokenize the text column to get the new column 'tokenized_text'
df_new['tokenized_text'] = [simple_preprocess(line, deacc=True) for line in df_new['discourse_text']] 
print(df_new['tokenized_text'].head(10))

In [6]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(df_new, test_size = 0.2, stratify = df_new['one_hot'])

In [7]:
x_train = train['tokenized_text']
y_train = train['discourse_type']
x_test = test['tokenized_text']
y_test = test['discourse_type']

In [8]:
from gensim.models import Word2Vec

In [9]:
dim = 200
w2v_model = Word2Vec(vector_size = dim)
w2v_model.build_vocab(x_train)
w2v_model.train(x_train, total_examples = w2v_model.corpus_count, epochs = 20)

In [10]:
#Build word vector for training set by using the average value of all word vectors in the tweet, then scale
def wordvector(text, size):
    vec = np.zeros(size).reshape((1, size))
    count = 0.
    for word in text:
        try:
            vec += w2v_model.wv[word].reshape((1, size))
            count += 1.
        except KeyError:
            continue
    if count != 0:
        vec /= count
    return vec

In [11]:
from sklearn.preprocessing import scale
train_vec = np.concatenate([wordvector(text, dim) for text in x_train])
train_vec = scale(train_vec)

In [12]:
w2v_model.train(x_test, total_examples =  len(x_test), epochs = w2v_model.epochs)

In [13]:
test_vec = np.concatenate([wordvector(text, dim) for text in x_test])
test_vec = scale(test_vec)

In [18]:
from sklearn import model_selection, svm
from sklearn import metrics

In [18]:
# word2vec svm
svm_clf = svm.SVC(kernel='rbf',verbose=True)
svm_clf.fit(train_vec, y_train)

In [19]:
svm_pred = svm_clf.predict(test_vec)

In [ ]:
metrics.accuracy_score(svm_pred, y_test)

In [15]:
# word2vec random forest
from sklearn.ensemble import RandomForestClassifier as RFC
rfc_clf = RFC(n_estimators = 100, oob_score = True)
rfc_clf.fit(train_vec, y_train)
rfc_pred = rfc_clf.predict(test_vec)
rfc_accuracy = metrics.accuracy_score(rfc_pred, y_test)

In [20]:
metrics.accuracy_score(rfc_pred, y_test)

In [23]:
from sklearn.preprocessing import MinMaxScaler
scaler1 = MinMaxScaler(copy = False)
scaler1.fit(train_vec)
scaler2 = MinMaxScaler(copy = False)
scaler2.fit(test_vec)
train_01 = scaler1.transform(train_vec)
test_01 = scaler2.transform(test_vec)

In [25]:
# word2vec naivebayes
from sklearn.naive_bayes import MultinomialNB
mnb = MultinomialNB()
mnb.fit(train_01,y_train)
mnb_pred = mnb.predict(test_01)
metrics.accuracy_score(mnb_pred, y_test)

In [ ]:
# dictionary for word count
from sklearn.feature_extraction.text import CountVectorizer
from nltk.tokenize import RegexpTokenizer
token = RegexpTokenizer(r'[a-zA-Z0-9]+')

In [ ]:
cv = CountVectorizer(stop_words = 'english', ngram_range = (1,1), tokenizer = token.tokenize)

In [ ]:
text_count = cv.fit_transform(df_new['discourse_text'])

In [ ]:
from sklearn.model_selection import train_test_split
x_train2, x_test2, y_train2, y_test2 = train_test_split(text_count, df_new['one_hot'], test_size = 0.2)

In [ ]:
# wordcount mnb
mnb2 = MultinomialNB()
mnb2.fit(x_train2,y_train2)
mnb2_pred = mnb2.predict(x_test2)

In [ ]:
metrics.accuracy_score(mnb2_pred, y_test2)

In [ ]:
# wordcount rf
rfc_clf2 = RFC(n_estimators = 100, oob_score = True)
rfc_clf2.fit(x_train2, y_train2)
rfc_pred2 = rfc_clf2.predict(x_test2)

In [ ]:
metrics.accuracy_score(rfc_pred2, y_test2)

In [ ]:
# wordcount svm
svm_clf2 = svm.SVC(verbose=True)
svm_clf2.fit(x_train2, y_train2)
svm_pred2 = svm_clf2.predict(x_test2)

In [ ]:
metrics.accuracy_score(svm_pred2, y_test2)